In [1]:
!git clone 'https://github.com/aakashvardhan/s8-normalization.git'

Cloning into 's8-normalization'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 66 (delta 25), reused 52 (delta 11), pack-reused 0
Receiving objects: 100% (66/66), 731.49 KiB | 2.89 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [3]:
import sys
sys.path.append('/content/s8-normalization')
sys.path.append('/content/s8-normalization/models')

In [5]:
from models.model import Net
from models.model_utils import model_summary
from config import get_config
config = get_config()
from utils import setup_cifar10_data

In [10]:
train_data,test_data,train_loader, test_loader = setup_cifar10_data(config)

CUDA Available? True


100%|██████████| 170498071/170498071 [00:10<00:00, 15924095.75it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
# Batch Norm
model = Net(config).to(config['device'])
model_summary(model,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
           Dropout-4           [-1, 16, 32, 32]               0
         ConvBlock-5           [-1, 16, 32, 32]               0
            Conv2d-6           [-1, 32, 32, 32]           4,608
              ReLU-7           [-1, 32, 32, 32]               0
       BatchNorm2d-8           [-1, 32, 32, 32]              64
           Dropout-9           [-1, 32, 32, 32]               0
        ConvBlock-10           [-1, 32, 32, 32]               0
           Conv2d-11           [-1, 16, 32, 32]             512
        MaxPool2d-12           [-1, 16, 16, 16]               0
  TransitionBlock-13           [-1, 16, 16, 16]               0
           Conv2d-14           [-1, 16,

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from utils import train, test

In [12]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, config['device'], train_loader, optimizer, epoch)
    test(model, config['device'], test_loader)

EPOCH: 0


Loss=1.125544548034668 Batch_id=390 Accuracy=45.46: 100%|██████████| 391/391 [00:16<00:00, 23.02it/s]



Test set: Average loss: 1.2234, Accuracy: 5584/10000 (55.84%)

EPOCH: 1


Loss=0.8551298379898071 Batch_id=390 Accuracy=60.69: 100%|██████████| 391/391 [00:16<00:00, 23.24it/s]



Test set: Average loss: 1.1008, Accuracy: 6033/10000 (60.33%)

EPOCH: 2


...


Test set: Average loss: 0.7954, Accuracy: 7256/10000 (72.56%)

EPOCH: 10


Loss=0.6561245918273926 Batch_id=390 Accuracy=79.74: 100%|██████████| 391/391 [00:16<00:00, 24.19it/s]



Test set: Average loss: 0.7367, Accuracy: 7468/10000 (74.68%)

EPOCH: 11


Loss=0.5053811073303223 Batch_id=72 Accuracy=81.04:  19%|█▊        | 73/391 [00:03<00:16, 18.86it/s]


KeyboardInterrupt: 